# Oil Well Selection Optimization

You work for the mining company OilyGiant. Your task is to find the best location for a new oil well.
Steps to choose the location:

- Collect parameters of oil wells in the selected region: oil quality and reserve volume
- Build a model to predict reserve volume in new wells
- Select oil wells with highest estimated values
- Choose the region with the highest total benefit for selected oil wells

You have data on crude samples from three regions. The parameters of each oil well in the region are already known. Create a model that helps choose the region with the highest profit margin. Analyze potential benefits and risks using bootstrapping technique.

# Data Description

- `id` — Unique identifier of an oil well
- `f0, f1, f2` — Three characteristics about the quality of the oil (the specific meaning is not important, but the characteristics themselves are significant)
- `product` — Volume of reserves in the oil well (thousands of barrels)

**Conditions**:

- Only linear regression is suitable for model training.
- When exploring the region, a study with 500 points is conducted and the best 200 points are selected for profit calculation.
- The budget for developing 200 oil wells is $100 million.
- One barrel of raw materials generates $4.5 in revenue. The revenue from one unit of product is $4500 (the volume of reserves is expressed in thousands of barrels).
- After risk assessment, only regions with a risk of loss below 2.5% should be kept. From those that meet the criteria, select the region with the highest average profit.

# Initialization

In [33]:
# Load libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Load data

In [34]:
# Load the data into DataFrames
df0 = pd.read_csv('../datasets/geo_data_0.csv')
df1 = pd.read_csv('../datasets/geo_data_1.csv')
df2 = pd.read_csv('../datasets/geo_data_2.csv')

# Prepare data

In [35]:
# Print the general/summary information about the DataFrame
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [36]:
# Print a random sample of 5 rows from the DataFrame
df0.sample(5)

,id,f0,f1,f2,product
4369,WooWm,-0.719937,0.801476,2.881326,129.752714
74776,FBKEu,-0.312182,1.039959,-1.962616,24.073036
58605,sSnlx,0.616175,-0.434014,0.682077,49.066313
10115,0fL4f,0.940062,0.538663,2.308844,56.114394
15668,clFDt,0.040309,0.328501,3.126859,76.162094


In [37]:
# Print the general/summary information about the DataFrame
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [38]:
# Print a random sample of 5 rows from the DataFrame
df1.sample(5)

,id,f0,f1,f2,product
50605,ssa5j,-5.336044,-6.075595,0.005342,3.179103
25548,sTo8R,8.488254,-1.601806,2.995629,80.859783
76289,KAnFV,8.550061,-4.716726,0.000137,0.000000
44443,CduiV,9.925779,3.377338,0.003386,0.000000
84023,a5xha,-3.451761,-11.279291,3.997681,110.992147


In [39]:
# Print the general/summary information about the DataFrame
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [40]:
# Print a random sample of 5 rows from the DataFrame
df2.sample(5)

,id,f0,f1,f2,product
98054,uQAZP,0.641425,-1.404748,9.935972,148.040922
7787,9ApAB,-1.768920,4.584891,0.273782,54.991991
50982,E776x,1.296762,-0.982589,4.164867,73.142485
61142,2S7yt,-0.099718,-1.951553,2.421158,106.587813
40185,8obx6,-1.902012,-1.096240,-3.489681,92.771181


## Observations
- The `id` column does not contain relevant information for model predictions.
- None of the three datasets contain missing values, and the data type of all columns is correct.
- All three datasets are ready to be used in model training.

## Remove unnecessary columns

In [41]:
# Remove the columns that will not be used in the analysis
df0 = df0.drop(['id'], axis=1)
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)

# Segment data

In [42]:
# Select a random seed for the experiments
random_state = np.random.RandomState(12345)

## Region 0

In [43]:
# Separate the data into features and target for the dataset 0
features0 = df0.drop(['product'], axis=1)
target0 = df0['product']

In [44]:
# Divide the data into training and validation subsets (75:25 ratio)
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(features0, target0, test_size=0.25, random_state=random_state)

## Region 1

In [45]:
# Separate the data into features and target for the dataset 1
features1 = df1.drop(['product'], axis=1)
target1 = df1['product']

In [46]:
# Divide the data into training and validation subsets (75:25 ratio)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1, test_size=0.25, random_state=random_state)

## Region 2

In [47]:
# Separate the data into features and target for the dataset 2
features2 = df2.drop(['product'], axis=1)
target2 = df2['product']

In [48]:
# Divide the data into training and validation subsets (75:25 ratio)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2, test_size=0.25, random_state=random_state)

# Model Training (Linear Regression)

## Region 0

In [49]:
# Test the Linear Regression model on dataset 0
model0 = LinearRegression()
model0.fit(features_train0, target_train0)
predicted_valid0 = model0.predict(features_valid0)
rmse0 = mean_squared_error(target_valid0, predicted_valid0) ** 0.5

# Save the predictions and correct answers for the validation set
predictions0 = pd.Series(predicted_valid0, index=target_valid0.index)

# Display on screen the average volume of forecasted reserves and the RMSE value
print('Average volume of projected reserves for region 0:', predicted_valid0.mean())
print('RMSE for region 0:', rmse0)

Average volume of projected reserves for region 0: 92.59256778438035
RMSE for region 0: 37.5794217150813


### Observations
- The average volume of the projected reserves for **`region 0`** is `92.59 thousand barrels`.
- The RMSE for **`region 0`** is `37.58`.

## Region 1

In [50]:
# Test the Linear Regression model on dataset 1
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)
rmse1 = mean_squared_error(target_valid1, predicted_valid1) ** 0.5

# Save the predictions and correct answers for the validation set
predictions1 = pd.Series(predicted_valid1, index=target_valid1.index)

# Display on screen the average volume of forecasted reserves and the RMSE value
print('Average volume of projected reserves for region 1:', predicted_valid1.mean())
print('RMSE for region 1:', rmse1)

Average volume of projected reserves for region 1: 68.76995145799752
RMSE for region 1: 0.8897367737680646


### Observations
- The average volume of the projected reserves for **`region 1`** is `68.77 thousand barrels`.
- The RMSE for **`region 1`** is `0.89`.

## Region 2

In [51]:
# Test the Linear Regression model on dataset 2
model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)
rmse2 = mean_squared_error(target_valid2, predicted_valid2) ** 0.5

# Save the predictions and correct answers for the validation set
predictions2 = pd.Series(predicted_valid2, index=target_valid2.index)

# Display on screen the average volume of forecasted reserves and the RMSE value
print('Average volume of projected reserves for region 2:', predicted_valid2.mean())
print('RMSE for region 2:', rmse2)

Average volume of projected reserves for region 2: 95.087528122523
RMSE for region 2: 39.958042459521614


### Observations
- The average volume of the projected reserves for **`region 2`** is `95.09 thousand barrels`.
- The RMSE for **`region 2`** is `39.96`.

## Intermediate Conclusion
- The highest average reserves are found in `region 2`, closely followed by `region 0`. However, it is notable that the **RMSE** is considerably high in `region 0 and 2`, indicating a higher level of error in the predictions of oil reserves in these regions. In contrast, `region 1` has a lower average reserve but also has a much lower **RMSE**, suggesting that the model's predictions are more accurate in this region.

# Profit Calculation

## Key values for calculations

In [52]:
# Budget for the development of 200 oil wells is $100 million
budget = 100_000_000

# The revenue from one unit of product is $4500 (the volume of reserves is expressed in thousands of barrels)
income_per_unit = 4_500

# Risk threshold of losses less than 2.5%
risk_threshold = 0.025

# Cost of development of a single well
cost_per_well = budget / 200
print('Cost of development of a single well:', cost_per_well)

# Volume of reserves sufficient to develop a new well without losses
sufficient_volume = cost_per_well / income_per_unit
print('Volume of reserves sufficient to develop a new well without losses:', sufficient_volume)

Cost of development of a single well: 500000.0
Volume of reserves sufficient to develop a new well without losses: 111.11111111111111


### Observations
- Development cost for a single well: `500,000`
- Volume of reserves sufficient to develop a new well without losses: `111.11`

## Sufficient Reserves Volume to Develop a New Well without Losses

### Region 0

In [53]:
print('Average volume of reserves for region 0:', target0.mean())

# Compare the obtained value with the average volume of reserves for region 0
if sufficient_volume > target0.mean():
    print('It is not possible to develop a new well in region 0 without losses.')
else:
    print('It is possible to develop a new well in region 0 without losses.')

Average volume of reserves for region 0: 92.50000000000001
It is not possible to develop a new well in region 0 without losses.


### Region 1

In [54]:
print('Average volume of reserves for region 1:', target1.mean())

# Compare the obtained value with the average volume of reserves for region 1
if sufficient_volume > target1.mean():
    print('It is not possible to develop a new well in region 1 without losses.')
else:
    print('It is possible to develop a new well in region 1 without losses.')

Average volume of reserves for region 1: 68.82500000000002
It is not possible to develop a new well in region 1 without losses.


### Region 2

In [55]:
print('Average volume of reserves for region 2:', target2.mean())

# Compare the obtained value with the average volume of reserves for region 2
if sufficient_volume > target2.mean():
    print('It is not possible to develop a new well in region 2 without losses.')
else:
    print('It is possible to develop a new well in region 2 without losses.')

Average volume of reserves for region 2: 95.00000000000004
It is not possible to develop a new well in region 2 without losses.


### Observations
- None of the three regions seem to have a high enough average volume to drill a new well without losses.
- Although `regions 0 and 2` have average volumes closest to what is needed to avoid losses, we must not forget that `region 1` has a lower RMSE.

## Expected total income from the 200 most profitable wells in each region

In [56]:
# Create a function to calculate the expected total income from the top 200 profitable wells for a region
def calculate_profit(actual_volumes, predicted_volumes):
    # Sort the predictions from highest to lowest
    sorted_predictions = predicted_volumes.sort_values(ascending=False)

    # Select the top 200 highest values
    selected = actual_volumes[sorted_predictions.index][:200]

    # Calculate the total expected profit
    total_revenue = (selected * income_per_unit).sum()
    
    return total_revenue - budget

### Region 0

In [57]:
# Calculate the expected total income from the 200 most profitable wells for the region 0
profit0 = calculate_profit(target_valid0, predictions0)
print('Expected total income from the 200 most profitable wells for the region 0:', profit0)

Expected total income from the 200 most profitable wells for the region 0: 33208260.43139851


### Region 1

In [58]:
# Calculate the expected total income from the 200 most profitable wells for the region 1
profit1 = calculate_profit(target_valid1, predictions1)
print('Expected total income from the 200 most profitable wells for the region 1:', profit1)

Expected total income from the 200 most profitable wells for the region 1: 24150866.966815084


### Region 2

In [59]:
# Calculate the expected total income from the 200 most profitable wells for the region 2
profit2 = calculate_profit(target_valid2, predictions2)
print('Expected total income from the 200 most profitable wells for the region 2:', profit2)

Expected total income from the 200 most profitable wells for the region 2: 25399159.458429456


### Observations
- `Region 0` has the highest expected total income out of the three, followed by `Region 2`, and then `Region 1`.

## Intermediate Conclusion
- Based on the results of the average volume in each region as well as the expected revenues, it is still not possible to determine which region is the best option for creating a new well.

# Risk and Profit Calculation for Each Region

## Bootstrapping with 1000 samples to find the distribution of profit

In [60]:
# Create a function to perform bootstrapping and calculate the distribution of benefits and risks for each region
def bootstrap_profit_and_loss(target, predictions, loop_size = 1_000, sample_size = 500):
    # Initialize the list of benefits
    values = []

    # Combine target and predictions into a DataFrame
    combined = pd.DataFrame({
        'target': target,
        'predictions': predictions
    })

    # Initialize the negative profit counter
    negative_profit_count = 0

    # Performing bootstrapping
    for _ in range(loop_size):
        # Sample the data
        subsample = combined.sample(sample_size, replace=True, random_state=random_state)

        # Separate target and predictions again
        subsample_target = subsample['target']
        subsample_predictions = subsample['predictions']

        # Calculate the profit of the sample and add it to the list of benefits
        profit = calculate_profit(subsample_target, subsample_predictions)
        values.append(profit)
        
        # Count the number of times the profit is negative
        if profit < 0:
            negative_profit_count += 1

    # Convert the list of benefits to a pandas Series
    values = pd.Series(values)

    # Calculate the average profit
    average_profit = values.mean()

    # Calculate the quantile of 2.5%
    lower = values.quantile(risk_threshold)

    # Calculate the quantile of 97.5%
    upper = values.quantile(1 - risk_threshold)

    # Evaluate loss risk
    loss_risk = negative_profit_count / loop_size

    return average_profit, loss_risk, lower, upper

In [61]:
# Create function to display the results of bootstrapping
def print_bootstrap_results(target_valid, predictions, region):
    average_profit, loss_risk, lower, upper = bootstrap_profit_and_loss(target_valid, predictions)

    print(f'95% confidence interval for the region {region}: [{lower:.1f}] -> [{upper:.1f}]')
    print(f'Average benefit for the region {region}: ${average_profit:.1f}')
    print(f'Loss risk for the region {region}: {loss_risk:.1%}')

### Region 0

In [62]:
# Bootstrapping with 1000 samples to find the profit distribution for region 0
print_bootstrap_results(target_valid0, predictions0, 0)

95% confidence interval for the region 0: [-761878.1] -> [9578465.3]
Average benefit for the region 0: $4238972.4
Loss risk for the region 0: 4.8%


### Region 1

In [63]:
# Bootstrapping with 1000 samples to find the profit distribution for region 1
print_bootstrap_results(target_valid1, predictions1, 1)

95% confidence interval for the region 1: [1080669.0] -> [9285744.4]
Average benefit for the region 1: $5132567.0
Loss risk for the region 1: 0.6%


### Region 2

In [64]:
# Bootstrapping with 1000 samples to find the profit distribution for region 2
print_bootstrap_results(target_valid2, predictions2, 2)

95% confidence interval for the region 2: [-1428006.3] -> [8933805.7]
Average benefit for the region 2: $3811203.6
Loss risk for the region 2: 7.4%


## Intermediate Conclusion
- Although the values obtained through bootstrapping may vary in each execution of the code, it is important to consider that these values remain within the same range.
- `Region 1` generally shows the highest average benefit and lowest risk of losses, indicating that this region has the greatest potential to achieve our objective.

# General Conclusion

During this project, a linear regression model was built to predict the volume of reserves in new oil wells based on geological data from three regions. Subsequently, a bootstrapping simulation was performed to estimate the distribution of potential benefits and risks in each region.

- An exploratory data analysis was conducted and a linear regression model was trained for each of the three regions. A linear regression model was fitted for each region using RMSE as the evaluation metric.
- Based on the model predictions, the top 200 wells with the highest predicted volumes of oil reserves were selected for drilling.
- To quantify uncertainty in the drilling decision and estimate potential profit, bootstrapping technique was employed. This provided a distribution of possible benefits and risk of losses.
- The bootstrapping analysis showed that `region 1` exhibits the `highest average benefit` as well as `lowest risk of losses`, suggesting it would be the most promising region for oil exploration.

Based on these results, it has been concluded that `region 1` is the most promising for future well drilling operations as it presents both `the highest average benefit` and `the lowest risk of losses`.